In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
import pickle
soc = TopSoc()

In [ ]:
fNyIn = soc.fsIn/2
print("fNyIn =",fNyIn)

In [ ]:
# Define the area to calibrate, based on an (arbitrary) mixer setting
fMixer = 0.75*soc.fsIn
print("fMixer =",fMixer)
soc.set_mixer(fMixer)
f0 = fMixer + 85.4
f1 = fMixer + 95.6
fTone = 0.5*(f0+f1)

In [ ]:
# Measure the nominal delay near fTone
importlib.reload(Scan)
scan = Scan.Scan(soc)
outCh = scan.soc.outFreq2ch(fTone)
print("measure nominal dealy for fTone=%f  outCh=%d"%(fTone, outCh))
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, doProgress=True, doPlot=True)

In [ ]:
# Make a calibration with nt transfers at nf frequencies, with the nominalDelay correction applied
nt,nf = 1,201
print("f0 =",f0)
print("f1 =",f1)
calibration = scan.makeCalibration(fMixer, f0, f1, nf=nf, nt=nt, doProgress=True,
                            nominalDelay=nominalDelay, verbose=True)
fscan = calibration['fscan']
print("delayApplied =",fscan['delayApplied'])
# For convenience, arrange these points as a single spectrum
spectrum = Scan.fscanToSpectrum(fscan)

In [ ]:
fList = scan.makeFList(fMixer, f0, f1,verbose=True)

In [ ]:
def iqPlot(fStart, fEnd):
    inds = (spectrum[0] > fStart ) & (spectrum[0] < fEnd)
    sx = spectrum[1]*np.exp(1j*spectrum[2])
    si = np.real(sx)
    sq = np.imag(sx)
    plt.plot(spectrum[0][inds],si[inds], ',', label="I")
    plt.plot(spectrum[0][inds],sq[inds], ',', label="Q")
    for f in fList:
        plt.axvline(f,color='r', alpha=0.2)
    plt.xlim((fStart,fEnd))
    plt.legend()
    plt.xlabel("frequency (MHz)")
    plt.ylabel("values [ADUs]")
fStart = 3164
fEnd = 3166
iqPlot(fStart, fEnd)

In [ ]:
def apPlot(fStart, fEnd):
    inds = (spectrum[0] > fStart ) & (spectrum[0] < fEnd)
    amp = spectrum[1]
    pha = spectrum[2]
    fig,ax = plt.subplots(2,1,sharex=True)
    
    ax[0].plot(spectrum[0][inds],amp[inds], ',')
    plt.plot(spectrum[0][inds],pha[inds], ',')
    for f in fList:
        ax[0].axvline(f,color='r', alpha=0.2)
        ax[1].axvline(f,color='r', alpha=0.2)
    ax[1].set_xlabel("frequency (MHz)")
    ax[0].set_ylabel("amplitude [ADUs]")
    ax[1].set_ylabel("phase [rad]")
    plt.xlim((fStart,fEnd))
apPlot(fStart,fEnd)

In [ ]:
fTones = np.array([3165.11])
amps = 0.9*np.ones(len(fTones))/len(fTones)

np.random.seed(1234991)
fis = np.random.uniform(0, 2*np.pi, size=len(fTones))
#print("warning in ipynb set fis all to zero")
#fis = np.zeros(len(fTones))
bandwidth = 0.5 # MHz
nf = 50
decimation = 2
nt = 1
doProgress = True
importlib.reload(Scan)
scan = Scan.Scan(soc)

fScanA = scan.fscan(fTones, amps, fis, bandwidth, nf, decimation,nt, doProgress=doProgress)
scan.applyDelay(fScanA, nominalDelay)

In [ ]:
def apPlotWithScan(fStart, fEnd, calibration, scan):
    cSpectrum = Scan.fscanToSpectrum(calibration['fscan'])
    cInds = (cSpectrum[0] > fStart ) & (cSpectrum[0] < fEnd)
    cAmp = cSpectrum[1]
    cPha = cSpectrum[2]

    sSpectrum = Scan.fscanToSpectrum(scan)
    sInds = (sSpectrum[0] > fStart ) & (sSpectrum[0] < fEnd)
    sAmp = sSpectrum[1]
    sPha = sSpectrum[2]

    
    fig,ax = plt.subplots(2,1,sharex=True)
    
    ax[0].plot(cSpectrum[0][cInds],cAmp[cInds], ',', label="calibration")
    ax[1].plot(cSpectrum[0][cInds],cPha[cInds], ',', label="calibration")
    ax[0].plot(sSpectrum[0][sInds],sAmp[sInds], ',', label="scan")
    ax[1].plot(sSpectrum[0][sInds],sPha[sInds], ',', label="scan")
    for f in fList:
        ax[0].axvline(f,color='r', alpha=0.2)
        ax[1].axvline(f,color='r', alpha=0.2)
    ax[1].set_xlabel("frequency (MHz)")
    ax[0].set_ylabel("amplitude [ADUs]")
    ax[1].set_ylabel("phase [rad]")
    
    #freqs,amps,fis = Scan.fscanToSpectrum(fscan)
    
    plt.xlim((fStart,fEnd))
apPlotWithScan(fStart,fEnd,calibration, fScanA)

In [ ]:
print(fScanA.keys())
print(calibration.keys())